In [43]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data
df = pd.read_csv('data/cleaned/merged_house_finance_data_2022.csv')

# drop Alaska because they know how to actually do elections (ranked choice voting)
df = df[df['state_po'] != 'AK']
df

,id,year,state,state_po,district,runoff,special,candidate_name,party,candidate_votes,total_votes,FEC_candidate_id,CAND_ICI,TTL_DISB
4,AL1,2022,ALABAMA,AL,1,False,False,JERRY L CARL,REPUBLICAN,140592,168150,H0AL01055,I,971793.34
5,AL1,2022,ALABAMA,AL,1,False,False,ALEXANDER M REMREY,LIBERTARIAN,26369,168150,NaN,NaN,NaN
6,AL2,2022,ALABAMA,AL,2,False,False,BARRY MOORE,REPUBLICAN,137460,198961,H8AL02171,I,430418.03
7,AL2,2022,ALABAMA,AL,2,False,False,PHYLLIS HARVEY-HALL,DEMOCRAT,58014,198961,H0AL02202,C,60219.66
8,AL2,2022,ALABAMA,AL,2,False,False,JONATHAN REALZ,LIBERTARIAN,3396,198961,H2AL02190,C,13281.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214,WY0,2022,WYOMING,WY,0,False,False,LYNNETTE GREYBULL,DEMOCRAT,47250,198198,H0WY01036,C,89155.39
1215,WY0,2022,WYOMING,WY,0,False,False,RICHARD BRUBAKER,LIBERTARIAN,5420,198198,H2WY01081,C,5000.00
1216,WY0,2022,WYOMING,WY,0,False,False,MARISSA JOY SELVIG,CONSTITUTION,4505,198198,H2WY01016,C,19936.16
1217,WY0,2022,WYOMING,WY,0,False,False,UNDERVOTES,NaN,3660,198198,NaN,NaN,NaN


In [44]:
df.dtypes

id                   object
year                  int64
state                object
state_po             object
district              int64
runoff                 bool
special                bool
candidate_name       object
party                object
candidate_votes       int64
total_votes           int64
FEC_candidate_id     object
CAND_ICI             object
TTL_DISB            float64
dtype: object

In [45]:
df['percent_of_vote'] = (df['candidate_votes'] / df['total_votes']).round(4)

In [46]:
# Calculate the difference between the highest and second highest percent_of_vote for each 'id'
df_sorted = df.sort_values(['id', 'percent_of_vote'], ascending=[True, False])
df_top2 = df_sorted.groupby('id').head(2)
df_diff = (-1 * df_top2.groupby('id')['percent_of_vote'].diff()).fillna(0).round(4)
df_margin = df_diff.groupby(df_top2['id']).last().reset_index()
df_margin.columns = ['id', 'vote_margin']

# Merge the vote margin back to the original dataframe
df_with_margin = df.merge(df_margin, on='id', how='left')

df_with_margin

,id,year,state,state_po,district,runoff,special,candidate_name,party,candidate_votes,total_votes,FEC_candidate_id,CAND_ICI,TTL_DISB,percent_of_vote,vote_margin
0,AL1,2022,ALABAMA,AL,1,False,False,JERRY L CARL,REPUBLICAN,140592,168150,H0AL01055,I,971793.34,0.8361,0.6793
1,AL1,2022,ALABAMA,AL,1,False,False,ALEXANDER M REMREY,LIBERTARIAN,26369,168150,NaN,NaN,NaN,0.1568,0.6793
2,AL2,2022,ALABAMA,AL,2,False,False,BARRY MOORE,REPUBLICAN,137460,198961,H8AL02171,I,430418.03,0.6909,0.3993
3,AL2,2022,ALABAMA,AL,2,False,False,PHYLLIS HARVEY-HALL,DEMOCRAT,58014,198961,H0AL02202,C,60219.66,0.2916,0.3993
4,AL2,2022,ALABAMA,AL,2,False,False,JONATHAN REALZ,LIBERTARIAN,3396,198961,H2AL02190,C,13281.19,0.0171,0.3993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1210,WY0,2022,WYOMING,WY,0,False,False,LYNNETTE GREYBULL,DEMOCRAT,47250,198198,H0WY01036,C,89155.39,0.2384,0.4286
1211,WY0,2022,WYOMING,WY,0,False,False,RICHARD BRUBAKER,LIBERTARIAN,5420,198198,H2WY01081,C,5000.00,0.0273,0.4286
1212,WY0,2022,WYOMING,WY,0,False,False,MARISSA JOY SELVIG,CONSTITUTION,4505,198198,H2WY01016,C,19936.16,0.0227,0.4286
1213,WY0,2022,WYOMING,WY,0,False,False,UNDERVOTES,NaN,3660,198198,NaN,NaN,NaN,0.0185,0.4286


In [47]:
df_with_margin.to_csv('output/temp.csv', index=False)

# adding in pvi and nhe

- pvi = partisan voting index
- nhe = national house environment

In [ ]:
# margin - PVI = adjusted_margin
# adjusted_margin - NHE = final_score
# compare final_score to TTL_DISB